In [1]:
import os
from dotenv import load_dotenv

# Best practice: Load environment variables from a .env file
load_dotenv()

# Securely get the API key from environment variables
# You must set this variable in your environment (e.g., in a .env file)
# The variable should be named GOOGLE_API_KEY
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    raise ValueError("GOOGLE_API_KEY environment variable not set. Please get your key from Google AI Studio.")

# 1. Import the correct class for Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI

# 2. Instantiate the Gemini model with the correct model name
llm = ChatGoogleGenerativeAI(
    google_api_key=google_api_key,
    model="gemini-2.5-flash",
    temperature=0
)

In [2]:
# Schema for structured output
from pydantic import BaseModel, Field
class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, justification="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")
print(output.search_query)
print(output.justification)

Calcium CT score and high cholesterol relationship
None


In [3]:
# Define a tool
def multiply(a: int, b: int) -> int:
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is 2 times 3?")

# Get the tool call
msg.tool_calls

[{'name': 'multiply',
  'args': {'b': 3.0, 'a': 2.0},
  'id': '9adccf98-3686-458d-b315-42070843a4c5',
  'type': 'tool_call'}]